# InertialVibrationAnalysis

## Description
This project is using for collect data from the ps5 controller and analyze the data to identify the status of the controller.
By using the inear resonant actuator with something certain signal and the inertial sensor to identify the status of the controller.

In [ ]:
### Data description
